In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import pandas as pd
import pystan
import numpy as np
import matplotlib.pyplot as plt
import pystan_utils
import os
import seaborn as sn
from  movie_recommendation_aux import *
from baselines import *

# Data preperation

In [ ]:
seed = 42
#dataset = 'ml-20m' #big
dataset = 'ml-latest-small' #small
ratings = pd.read_csv(os.path.join(dataset,'ratings.csv'))
# HACK -- small movies.csv is apararently missing movies from small ratings.csv
movies = pd.read_csv(os.path.join('ml-20m','movies.csv')) 
#Create like column
ratings['like'] = (ratings.rating >= 3.0 )+ 0

#Convert ratings from half step stars, to 1-10 stars
ratings_dict = {j:i+1 for i,j in enumerate(sorted(ratings['rating'].unique()))}
ratings['rating'] =  ratings['rating'].apply(lambda rating: ratings_dict[rating])

Data samples

In [ ]:
ratings.head()

For now instead of multiclass classification based on stars, turn problem into binary classification by defining 'like' for all movies rated 3.0 stars or above, and 'not-like' for all movies below 3.0.

For each user split sample (without replacement) 90% of data as training data and the remaining 10% as validation data. As some movies might never be sampled in the training set, remove those films from the validation set.

In [ ]:
val_size = 0.3
#For sample randomly the validation set (note some movies might never be sampled)
val_set = ratings.groupby('userId').apply(lambda g: g.sample(frac=val_size,random_state=seed))
val_set.index =  val_set.index.droplevel()
#train set is compliment of val_set
train_set = ratings[~ratings.isin(val_set).all(1)]
#Possibly remove movies from validation set that was never sampled in the dataset
val_set = val_set[val_set.movieId.isin(train_set.movieId)] 

Make sure that not too many samples was removed. Fraction of the removed data is:

In [ ]:
(len(ratings) - (len(train_set)+len(val_set)))/len(ratings)

As the movieIds does not necesarrily correspond to integer indices, make new ids such that they can be used as indices in stan vectors/matrixes:

In [ ]:
unique_keys = train_set.movieId.unique()
indices = range(1,len(unique_keys)+1)
movie_id_dict = dict(zip(unique_keys, indices ))
id_movie_dict = dict(zip(indices, unique_keys))
train_set['movieIdNoHoles'] = train_set['movieId'].apply(lambda movie_id: movie_id_dict[movie_id])
val_set['movieIdNoHoles'] = val_set['movieId'].apply(lambda movie_id: movie_id_dict[movie_id])

# Model

### Generative Process

```
for (userId, movieId) in [(u1,m1),(u2,m2),...,(uN,mN)]
    affinity = 0;
    for (t in 1:num_traits){
        traitAffinity = trait[movieId, t] * preference[userId, t];
        affinity += traitAffinity
    generate prediction such that prediction ~ bernoulli_logit(affinity);
```

Probably summing the trait affinities and using the affinity as logit is not the way to discrimitate between likes.

### PGM

![alt text](figs/PGM.png "Title")
*PGM of model. We use the notation of http://www.mbmlbook.com that specifices the pgm as a bipartite graph where the squares explicitely denotes the distribution* 

### STAN

In [ ]:
model_definition = """ data {
    int num_movies;             // number of data items
    int num_traits;
    int num_users;  
    
    int num_likes;

    
    int likes_obs[num_likes];
    int userId_obs[num_likes];
    int movieId_obs[num_likes];
    
    int num_missing;
    int userId_missing [num_missing];
    int movieId_missing [num_missing];
    
}
parameters {
    matrix[num_movies,num_traits] trait;
    matrix[num_users ,num_traits] preference;
    vector[num_movies] trait_bias;
    vector[num_users] preference_bias;
    
} 

model {
    real affinity;
    //fix symmetries
    /*for(i in 1:num_traits){
        for( j in 1:num_traits){
            if(i == j){
                trait[i,j] ~ normal(1,0.0001);
            }
            else{
                trait[i,j] ~ normal(0,0.0001);
            }
        }
    }
    */
    for (n in 1:num_likes){
        affinity = 0;
        trait_bias[movieId_obs[n]] ~ normal(0,10);
        preference_bias[userId_obs[n]] ~ normal(0,10);
        for (t in 1:num_traits){
            preference[userId_obs[n], t] ~ normal(0,10);
            //if (movieId_obs[n] > num_traits){
                trait[movieId_obs[n], t] ~ normal(0,10);
            //}
            affinity += trait[movieId_obs[n], t]*preference[userId_obs[n], t];
            
        }
        affinity += trait_bias[movieId_obs[n]] + preference_bias[userId_obs[n]];
        
        likes_obs[n] ~ bernoulli_logit(affinity);

    }
}

generated quantities {
    real<lower=0, upper=1> predictions[num_missing];
    
    for(i in 1:num_missing){
        real affinity = 0;
        for (t in 1:num_traits){
            affinity += trait[movieId_missing[i], t] * preference[userId_missing[i], t];
        }
        affinity += trait_bias[movieId_missing[i]] + preference_bias[userId_missing[i]];
        predictions[i] = bernoulli_logit_rng(affinity);
    }
}
"""

In [ ]:
%%time
# create Stan model object
sm = pystan.StanModel(model_code = model_definition)

# Sanity checks

### Simple data set
Generate a simple data containing two groups (p1,p2) of people and to groups of movies (m1,m2).
* p1 likes all movies in m1 but dislikes all movies in m2.
* p2 likes all movies in m2 but dislikes all movies in m1.

In [ ]:
train_set_fake, val_set_fake = generate_fake_data(val_size=0.3, seed=seed)

In [ ]:
data_fake, num_users_fake, num_movies_fake = generate_data_dict(train_set_fake, val_set_fake,n_traits=2)

In [ ]:
%%time
#sampling takes forever here, but VB seems to work really well
#fit = sm.sampling(data=data, iter=100, algorithm="NUTS", chains=1, seed=seed, verbose=True)
fit_fake = sm.vb(data=data_fake,seed=seed)

In [ ]:
predictions_fake, probabilities_fake = pystan_utils.vb_extract_predictions(fit_fake)
get_precision(predictions_fake,val_set_fake)

The precision is 1.0 which shows that we can predict the two groups accurately.

In [ ]:
preferences_fake=pystan_utils.vb_extract_variable(fit_fake, 'preference[', var_type='matrix', dims=[num_users_fake,2])
traits_fake=pystan_utils.vb_extract_variable(fit_fake, 'trait[', var_type='matrix', dims=[num_movies_fake,2])

In [ ]:
plt.figure()
plt.scatter(preferences_fake[:,0], preferences_fake[:,1],label='preference')
plt.scatter(traits_fake[:,0], traits_fake[:,1], label='traits')
plt.legend()

The preference and traits are nicely separated.

# Baseline
As a baseline implimentation we use the 

In [ ]:
print("Running Damped Baseline with beta=0...", end="")
model = DampedUserMovieBaselineModel(damping_factor=0)
#validator = PerformanceOverTimeValidator(model, n_year_period=2)
#years_4, errs_4 = validator.validate(ratings[['userId', 'movieId']], ratings['rating'], ratings['year'])
#print("Done!")
model.fit(train_set[['userId', 'movieId']], train_set['rating'])
baseline_prediction = model.predict(val_set[['userId', 'movieId']])
get_NDCG(baseline_prediction, val_set, k=10)

# Results

In [ ]:
data, num_users, num_movies = generate_data_dict(train_set, val_set,n_traits=2)

In [ ]:
%%time
#sampling takes forever here, but VB seems to work really well
#fit = sm.sampling(data=data, iter=100, algorithm="NUTS", chains=1, seed=seed, verbose=True)
fit = sm.vb(data=data,seed=seed)

In [ ]:
predictions, probabilities = pystan_utils.vb_extract_predictions(fit)
get_precision(predictions,val_set)

The precision is larger than random.

In [ ]:
get_NDCG(probabilities,val_set, k=5)

## Show preferences
Here the latent traits and preferences 

In [ ]:
preferences=pystan_utils.vb_extract_variable(fit, 'preference[', var_type='matrix', dims=[num_users,2])
traits=pystan_utils.vb_extract_variable(fit, 'trait[', var_type='matrix', dims=[num_movies,2])

In [ ]:
plt.figure()
plt.scatter(preferences[:,0], preferences[:,1])

In [ ]:
plt.figure()
plt.scatter(traits[:,0], traits[:,1])

Lets plot some extreme values of trait0. We would expect to see that the trait is discriminating between films using a latent trait of the film.

In [ ]:
n_extreme = 10
sorted_trait_0_ids = np.argsort(traits[:,0])
lowest_ids = sorted_trait_0_ids[:n_extreme]+1
highest_ids = sorted_trait_0_ids[-n_extreme:]+1
lowest_movie_ids = [ id_movie_dict[lowest_id] for lowest_id in lowest_ids]
highest_movie_ids = [ id_movie_dict[highest_id] for highest_id in highest_ids]

In [ ]:
movies[movies.movieId.isin(lowest_movie_ids)]

In [ ]:
movies[movies.movieId.isin(highest_movie_ids)]

By visual inspection of the low/high scoring, we cannot really see any latent trait that is used for discriminating.

Similar films should have similiar trait values. Therefore LOTR movies are inspected. 

In [ ]:
ids_lotr = [movie_id_dict[movie]-1 for movie in  movies[movies.title.str.contains('Lord of the Rings')].movieId]
plt.figure()
plt.scatter(traits[:,0], traits[:,1])
plt.scatter(traits[ids_lotr][:,0],traits[ids_lotr][:,1])

Plot the movies for which the porsterior of the traits has low variance

In [ ]:
plot_low_variance_movies(fit,movies,num_movies,id_movie_dict)

Plotting traits and biases for some movies to see what dominates what.

In [ ]:
samples_var,means_var, stds_var, names_var=pystan_utils.vb_extract(fit)
plt.figure()
for i in range(1,4):
    sn.kdeplot(samples_var[f'trait_bias[{i}]'], color='b')
    sn.kdeplot(samples_var[f'trait[{i},1]'], color='r')
    sn.kdeplot(samples_var[f'trait[{i},2]'], color='k')

It is seen that in some cases the bias is close to zero and sometimes not.

In [ ]:
model_definition_genre = """ data {
    int num_movies;             // number of data items
    int num_traits;
    int num_users;  
    
    int num_likes;

    
    int likes_obs[num_likes];
    int userId_obs[num_likes];
    int movieId_obs[num_likes];
    
    int num_missing;
    int userId_missing [num_missing];
    int movieId_missing [num_missing];
    
    matrix[num_likes,20] genre;
    
}
parameters {
    matrix[num_movies,num_traits] trait;
    matrix[num_users ,num_traits] preference;
    vector[num_movies] trait_bias;
    vector[num_users] preference_bias;
    matrix[20,num_traits] genre_weight;
    vector[num_traits] genre_bias;
    
} 

model {
    real affinity;
    row_vector[num_traits] traitMean;
    
    genre_bias ~ normal(0,2);
    
    for (t in 1:num_traits){
        genre_weight[t] ~ normal(0,2);
    }
    
    
    

    for (n in 1:num_likes){
        affinity = 0;
        
        traitMean = genre[n]*genre_weight;
        
        trait_bias[movieId_obs[n]] ~ normal(0,10);
        preference_bias[userId_obs[n]] ~ normal(0,10);
        
        
        for (t in 1:num_traits){
            
            preference[userId_obs[n], t] ~ normal(0,10);
            trait[movieId_obs[n], t] ~ normal(traitMean[t] ,10);

            affinity += trait[movieId_obs[n], t]*preference[userId_obs[n], t];
            
        }
        affinity += trait_bias[movieId_obs[n]] + preference_bias[userId_obs[n]];
        
        likes_obs[n] ~ bernoulli_logit(affinity);

    }
}

generated quantities {
    real<lower=0, upper=1> predictions[num_missing];
    
    for(i in 1:num_missing){
        real affinity = 0;
        for (t in 1:num_traits){
            affinity += trait[movieId_missing[i], t] * preference[userId_missing[i], t];
        }
        affinity += trait_bias[movieId_missing[i]] + preference_bias[userId_missing[i]];
        predictions[i] = bernoulli_logit_rng(affinity);
    }
}
"""

In [ ]:
%%time
# create Stan model object
sm_genre = pystan.StanModel(model_code = model_definition_genre)

In [ ]:
data, num_users, num_movies = generate_data_dict(train_set, val_set,n_traits=16)

In [ ]:
%%time
#sampling takes forever here, but VB seems to work really well
#fit = sm.sampling(data=data, iter=100, algorithm="NUTS", chains=1, seed=seed, verbose=True)
fit_genre = sm_genre.vb(data=data,seed=seed)

In [ ]:
predictions_genre, probabilities_genre = pystan_utils.vb_extract_predictions(fit_genre)
get_precision(predictions_genre,val_set)

The precision is larger than random.

In [ ]:
get_NDCG(probabilities_genre,val_set, k=5)

In [ ]:
samples_var,means_var, stds_var, names_var=pystan_utils.vb_extract(fit_genre)

In [ ]:
plt.figure()
for i in range(1,20+1):
    sn.kdeplot(samples_var[f'genre_weight[{i},1]'], color='b')
    sn.kdeplot(samples_var[f'genre_weight[{i},2]'], color='r')
    

# Questions

* *Expectation Propagation vs. Variational Bayes (10% difference in precision)?*
* *Does these values make sense (low ELBO)?*
```
Begin stochastic gradient ascent.
  iter             ELBO   delta_ELBO_mean   delta_ELBO_med   notes 
   100     -3124403.404             1.000            1.000
   200     -1131926.528             1.380            1.760
   300     -1068337.903             0.940            1.000
   400     -1037532.838             0.712            1.000
   500      -982924.262             0.581            0.060
   600     -1024784.262             0.491            0.060
   700      -967588.749             0.429            0.059
   800      -966833.896             0.376            0.059
   900      -954641.153             0.335            0.056
  1000      -932961.912             0.304            0.056
  1100      -933640.840             0.204            0.041
  1200      -944417.031             0.029            0.030
  1300      -933809.949             0.025            0.023
  1400      -922032.712             0.023            0.013
  1500      -929080.979             0.018            0.013
  1600      -923651.819             0.015            0.011
  1700      -926248.001             0.009            0.011   MEAN ELBO CONVERGED
 ```
* *Should we care about symmetries?* 
```samples_dict, means_dict, var_names = pystan_utils.vb_extract(fit2)
pystan_utils.plot_kde(samples_dict['trait[1,1]'])
```
* *Why do we not get the same values for lotr in the plots*?

# Modelling stars

To model star ratings, instead of likes we need to change our model from a binary classification model, to a multivariate one.

This can be done by simply replacing the bernoulli distribution with a categorical to generate outputs between 1-10.
Now we however need to model the parameters of the categorical distribution. Our affinity variable is just a number, so a naive way would be to define parameters $\beta$ as 

$$
\mathbf{\beta}_c = [\beta_1, \beta_2, \dots , \beta_c] = w^T\text{affinity}
$$

where $w\in \mathbb{R}^C$


$$
star_{u,m} \sim categorical(star_{u,m} |\text{softmax}( \mathbf{w}^T\text{affinity}_{u,m}))
$$

In [ ]:
model_definition = """ data {
    int num_movies;             // number of data items
    int num_traits;
    int num_users;  
    
    int num_likes;

    
    int stars_obs[num_likes];
    int userId_obs[num_likes];
    int movieId_obs[num_likes];
    
    int num_missing;
    int userId_missing [num_missing];
    int movieId_missing [num_missing];
    
}
parameters {
    matrix[num_movies,num_traits] trait;
    matrix[num_users ,num_traits] preference;
    vector[num_movies] trait_bias;
    vector[num_users] preference_bias;
    vector[10] w;
    
} 

model {
    real affinity;
    w ~normal(0,10);
    for (n in 1:num_likes){
        affinity = 0;
        trait_bias[movieId_obs[n]] ~ normal(0,10);
        preference_bias[userId_obs[n]] ~ normal(0,10);
        for (t in 1:num_traits){
            preference[userId_obs[n], t] ~ normal(0,10);
 
            affinity += trait[movieId_obs[n], t]*preference[userId_obs[n], t];
            
        }
        affinity += trait_bias[movieId_obs[n]] + preference_bias[userId_obs[n]];
        
        stars_obs[n] ~ categorical_logit(affinity * w);

    }
}

generated quantities {
    int predictions[num_missing];
    
    for(i in 1:num_missing){
        real affinity = 0;
        for (t in 1:num_traits){
            affinity += trait[movieId_missing[i], t] * preference[userId_missing[i], t];
        }
        affinity += trait_bias[movieId_missing[i]] + preference_bias[userId_missing[i]];
        predictions[i] = categorical_logit_rng(affinity*w);
    }
}
"""

In [ ]:
%%time
# create Stan model object
sm_stars = pystan.StanModel(model_code = model_definition)

In [ ]:
data, num_users, num_movies = generate_data_dict(train_set, val_set,n_traits=2, stars=True)

In [ ]:
%%time
#sampling takes forever here, but VB seems to work really well
#fit = sm.sampling(data=data, iter=100, algorithm="NUTS", chains=1, seed=seed, verbose=True)
fit_stars = sm_stars.vb(data=data,seed=seed)

In [ ]:
predictions_stars, probabilities_stars = pystan_utils.vb_extract_predictions(fit_stars, multi_class=True)
get_precision(predictions_stars,val_set)

In [ ]:
pystan_utils.vb_extract_variable(fit_stars, 'w', 'vector')

In [ ]:
#Doesn't work for multiclass output at the momement
get_NDCG(probabilities_stars,val_set, k=5)